# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](https://networkx.org/documentation/networkx-2.4/tutorial.html). For this homework, you may only use the basic undirected graph methods listed [here](https://networkx.org/documentation/networkx-2.4/reference/classes/graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [1]:
import networkx
homer = open('homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

Next implement a function to read in the edges from the input file.

In [2]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A list of the nodes in the graph of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: implement function
    #Initialize empty list to store nodes
    nodes=[]
    #Loop through the file line by line
    for line in gfile:
        #If there is a line with length greater than or equal to 2 and the character in index 2 is a '  ' (character id assignments)
        if line is not None and len(line) >= 2 and line[2] == ' ':
            #A given character id will be the first 2 items in the line
            char_id=line[:2]
            #Add this to a list of character id nodes
            nodes=nodes+[char_id]
    return(nodes)
    pass

In [3]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        The edges in the graph as a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    #return to the first line of the file
    gfile.seek(0)
    #Initialize an empty list to store edges
    edges=[]
    #Loop through the file line by line
    for line in gfile: 
        #Place the full line as a string into a list 
        data=line.splitlines()
        #Iterate through that line
        for line in data:
            #If the line is not empty and the item in index 2 is not equal to ' ' and index 0 is not equal to "*" (not character id assignments or notes)
            if len(line) != 0 and line[2] != ' ' and line[0] != "*":
                #Start at index 2 or 3 depending if the chapter is a single number of two numbers
                if line[1]==":": start_index=2
                else: start_index=3
                #Split the line first by commas between characters and then split again on semicolons for chunks/groups of characters
                char_groups=[x.split(",") for x in line[start_index:].split(";")]
                #For each group of characters that interacted
                for group in char_groups:
                    #For every character that appears in that group
                    for first_char in range(len(group)):
                        #For every other character in that group
                        for other_char in range(first_char+1,len(group)):
                            #Add to the list of edges this edge between the characters
                            edges=edges+[(group[first_char],group[other_char])]
    return(edges)
    pass

The following code should now correctly create the graph.

In [4]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [5]:
def Search(graph, root):
    """Runs Search from vertex root in a graph. Neighboring nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    # DFS calls search recursively until it finds all nodes reachable from the source
    def DFS(graph, source):
        #Initialize empty dictionary explored to indicate whether each node has been visited
        explored={}
        #Initialize explored to 0 for all nodes
        for u in graph.nodes:
            explored[u]=0
        #if source node is not in the graph, return empty list
        if source not in graph.nodes(): 
            return([])
        #Mark source node explored
        explored[source]=1
        #Initialize an empty stack to store nodes in the order they were first visited
        stack=[]
        #Call search recursively until all reachable nodes have been discovered
        explored,stack = search(source,explored,graph,stack)
        #Return the stack of visited nodes
        return(stack)
    
    # search is called within DFS to explore all neighbors of the node in alphabetical order
    def search(u,explored,graph,stack):
        #If a node is discovered, mark explored = 1 so it won't be visited again
        explored[u]=1
        #Add the node to the preorder traversal stack 
        stack=stack+[u]
        #For every neighbor of that node, visit them alphabetically
        for v in sorted(list(graph.neighbors(u))):
            #If they are unexplored, explore them by calling search()
            if explored[v]==0:
                explored,stack=search(v,explored,graph,stack)
        #Return the explored array and stack of visited nodes
        return(explored,stack)
    
    return(DFS(graph, root))
    pass   

We will check the correctness of your code by verifying that it correctly computes the connected component of node Ulysses (node `OD`).

In [6]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [7]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    # DFS calls search recursively until it finds all connect components of the graph
    def DFS(graph):
        #Initialize empty dictionary explored to indicate whether each node has been visited
        explored={}
        #Initialize empty list to store connected components
        conn_comps=[]
        #Initialize explored to 0 for all nodes
        for u in graph.nodes:
            explored[u]=0
        #Sort nodes alphabetically
        nodes=sorted(list(graph.nodes))
        #For every node in the graph
        for u in nodes:
            #Initialize an empty stack to hold all nodes reachable from u
            stack=[]
            #Call search recursively until all reachable nodes have been discovered
            if explored[u]==0:
                  explored,stack = search(u,explored,graph,stack)
            #If the stack isn't empty, add it to the list of connected components
            if len(stack)>0:
                conn_comps=conn_comps+[stack]
        #Return list of connected components
        return(conn_comps)
    
    # search is called within DFS to explore all neighbors of the node in alphabetical order
    def search(u,explored,graph,stack):
        #If a node is discovered, mark explored = 1 so it won't be visited again
        explored[u]=1
        #Add the node to the preorder traversal stack 
        stack=stack+[u]
        #For every neighbor of that node, visit them alphabetically
        for v in sorted(list(graph.neighbors(u))):
            #If they are unexplored, explore them by calling search()
            if explored[v]==0:
                explored,stack=search(v,explored,graph,stack)
        #Return the explored array and stack of visited nodes
        return(explored,stack)
    
    return(DFS(graph))
    pass

We will check correctness of your code by verifying that your output is identical to our solution.

In [8]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [9]:
component_sizes = [len(c) for c in character_interactions]
print("There are 12 connected components in the Iliad:{}".format(len(component_sizes) == 12))
print("The giant component has size 542:{}".format(max(component_sizes) == 542))
print("There are 5 isolated characters:{}".format(len([c for c in component_sizes if c == 1]) == 5))

There are 12 connected components in the Iliad:True
The giant component has size 542:True
There are 5 isolated characters:True
